In [ ]:
%matplotlib inline
%run ../setup/nb_setup

# Orbits 3: Orbits in Triaxial Potentials

Author(s): Adrian Price-Whelan


## Learning goals

In this tutorial, we will introduce triaxial potential models, and explore the additional complexity that this brings to the landscape of orbits, as compared to orbits in axisymmetric potential models.


## Introduction

In the last tutorial, we saw that reducing the symmetry of a potential from spherical to axisymmetric reduced the number if isolating integrals of motion from four to three. Will removing another potential symmetry — going from axisymmetric to triaxial — further reduce the number of isolating integrals? Luckily no: It turns out that regular orbits in triaxial potentials still have three isolating integrals. However, the orbit structure of triaxial potentials is more complex: A new type of orbit family can exist in triaxial potentials known as "box orbits," and tube orbits (what we are familiar with from axisymmetric models) can only exist around either the short or long axis of the potential. Triaxial potentials also generally have larger regions of chaotic orbits, making chaos more relevant in triaxial mass models.

In this tutorial, we will introduce some triaxial gravitational potential models, demonstrate the different types of orbits that can exist in triaxial models, and compare chaotic and regular orbits.


## Terminology and Notation

- (See Orbits tutorials 1 and 2)

### Notebook Setup and Package Imports

In [ ]:
from astropy.constants import G
import astropy.units as u

from IPython.display import HTML
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

## Triaxial Potential Models

To recap the last tutorial, we found that we could define axisymmetric potential–density pairs by replacing the spherical radius with a elliptical radius that scales the $z$ axis differently. Axisymmetric models are useful for defining potentials for flattened mass distributions, such as from galaxy disks. Triaxial potential models are models in which the isopotential or isodensity contours coincide with ellipsoidal surfaces. These models are useful for representing the outskirts of dark matter halos (which have been found to be triaxial, to first order) and for representing the stellar distributions in elliptical galaxies. 

To create a triaxial potential, we can  instead replace the spherical radius with an ellipsoidal radius defined as

$$
r^2_{\rm ell} = \frac{x^2}{a^2} + \frac{y^2}{b^2} + \frac{z^2}{c^2}
$$

where often by convention, $a$ is set to $a=1$ so that $b$ is the $y/x$ *axis ratio* and $c$ is the $z/x$ axis ratio. As with the axisymmetric models, we can either replace the radius in the expression of a spherical potential, or replace the radius in the expression of the density function, but these have different effects. Ideally, we would want to replace the radius in the form of a spherical density with the ellipsoidal radius, because then the density would be guaranteed to be positive and physical at all positions. However, there are very few triaxial solutions to Poisson's equation known, so most triaxial density distributions do not have analytic expressions for the potential (or gradients, which we need to compute orbits). It is therefore common to instead replace that radius in a potential expression with an ellipsoidal radius, but this has the same caveat as we saw with axisymmetric models that for some choices of $b, c$ the density can become negative. 


### Example: The Triaxial Log Potential

One example of such a potential model is the triaxial extension of the flattened log potential model discussed in the previous tutorial. The form of the potential is given by

$$
\Phi_L(\boldsymbol{x}) = \frac{1}{2}\,v_0^2 \, \ln\left(\frac{x^2}{q_1^2} + \frac{y^2}{q_2^2} + 
    \frac{z^2}{q_3^2} + r_h^2 \right)
$$

where the axis ratios are given by the parameters $(q_1, q_2, q_3)$, and by convention we typically set $q_1=1$.

With Gala, we can define a triaxial log potential by specifying axis ratio parameters `q1`, `q2`, `q3`.

### Exercise: Define a Triaxial Log Potential with Gala

Define a Gala `LogarithmicPotential` object with:
* $v_c=230~\textrm{km}~\textrm{s}^{-1}$
* $r_h=15~\textrm{kpc}$
* $q_1 = 1$
* $q_2 = 0.9$
* $q_3 = 0.8$

In [ ]:
# Define the potential object here
# triaxial_log =

## Integrals of Motion and Orbit Classes in Triaxial Models

Regular orbits in triaxial potential models still have three integrals of motion, however, like in the axisymmetric case, these are again not easily associated with classical integrals or analytic expressions. Regular orbits do have actions, but the interpretation of the actions often depends on the type of orbit being studied. To understand this a bit more, we need to look at the different types of orbit families that exist in triaxial potential models. Recall that in spherical models, all orbits are planar, and generic orbits form rosette patterns because of radial and azimuthal oscillations that often have an irrational frequency ratio. In axisymmetric models, orbits conserve the $z$ component of their angular momentum and so preserve their circulation around the symmetry axis of the potential, forming vertically-thickened analogs to the planar rosette patterns called "tube" orbits. In triaxial models, there are four general classes of orbits: short-axis tube orbits, "inner" long-axis tube orbits, "outer" long-axis tube orbits, and box orbits. 


## Tube Orbits

Tube orbits in triaxial models can only exist around the long and short axes of the potential, but around these axes the orbits look a lot like their axisymmetric counterparts. In the exercise below, we will compute a few tube orbits and take a closer look at their properties.


### Exercise: Long- and short-axis tube orbits

Define a `PhaseSpacePosition` object to represent two initial conditions: 

* At $\boldsymbol{x} = (10, 1, 3)~\textrm{kpc}$, with $v_y = v_c$
* At $\boldsymbol{x} = (3, 10, 1)~\textrm{kpc}$, with $v_z = v_c$

(other velocity components set to 0)

In [ ]:
# Define the initial conditions here
# tube_w0s = ...

Integrate these orbits in the `triaxial_log` potential defined above for a total integration time of 100 Gyr with a timestep of 2 Myr:

In [ ]:
# Integrate the orbits here:
# tube_orbits = ...

Plot the two orbits in all 2D projections of the 3D positions (x-y, x-z, y-z) on separate figures:

Compute and plot the angular momentum components for the two orbits as a function of time:

In [ ]:
# tube_orbits_L = ...

What differences do you see in the time-series angular momentum components as compared to orbits in an axisymmetric model?

Answer here: ...

**Bonus plots**: Some animations showing the 3D structure of these orbits

In [ ]:
azims = np.linspace(0, 180, 128)
half_elevs = np.linspace(-80, 80, len(azims) // 2)
elevs = np.concatenate((half_elevs, half_elevs[::-1]))

fig, anim = animate_3d(tube_orbits[:, 0], azims=azims, elevs=elevs, total_time=8)
plt.close(fig)
HTML(anim.to_html5_video())

In [ ]:
fig, anim = animate_3d(tube_orbits[:, 1], azims=azims, elevs=elevs, total_time=8)
plt.close(fig)
HTML(anim.to_html5_video())

### Exercise: Tube orbits around the intermediate axis?

Set up initial conditions to compute a tube orbit around the intermediate axis, starting from the position $\boldsymbol{x} = (10, 0.5, 0)~\textrm{kpc}$.

In [ ]:
# y_tube_w0 = ...

Integrate this orbit for the same time array as the tube orbits we computed above:

In [ ]:
# y_tube_orbit = ...

Plot the orbit in projections:

Compute the angular momentum components and plot them:

What is different about the angular momentum component time series for this orbit?

## Box Orbits

As mentioned above, triaxial potentials host another class of orbits that are distinct from tube orbits: *Box orbits*. These orbits do not maintain a fixed sense of rotation or circulation around an axis and so they fill a boxy volume in position space over time. Box orbits tend to have initial conditions without much tangential velocity, for example:

In [ ]:
box_orbit1_w0 = gd.PhaseSpacePosition(
    pos=[12, 4.0, 4.0] * u.kpc, vel=[0, 0, 0.0] * u.km / u.s
)

box_orbit1 = triaxial_log.integrate_orbit(
    box_orbit1_w0, t=tube_orbits.t, Integrator=gi.DOPRI853Integrator
)

_ = box_orbit1.plot_3d()

Because they do not maintain a sense of circulation around any of the axes, we can guess that none of the angular momentum components are even approximately conserved: We expect that all three components likely change sign over the course of the orbit. For the orbit we computed above, we can see that this is true:

In [ ]:
plt.plot(box_orbit1.t.value, box_orbit1.angular_momentum().T)
plt.xlim(0, 10000)

Because these orbits look nothing like tube orbits, and so there is no sense in which the epicycle approximation makes sense here, the actions for box orbits have a different interpretation than the actions for tube orbits (which can be understood as radial, azimuthal, and vertical actions like in the axisymmetric case). For box orbits, the actions do not have an exact physical interpretation: they will be quantifications of the amplitudes of independent oscillations. However, to first order, if the potential model is aligned along the coordinate axes, conceptually they will be related to the amplitude of oscillation in the $x$, $y$, and $z$ directions. Actions for box orbits are typically referred to as 

$$
\boldsymbol{J} = (J_1, J_2, J_3)
$$

to emphasize generality, i.e. that there is no direct or precise correspondence to coordinates.

### Exercise: A grid of orbits with equal energy

In this exercise, we are going to compute a grid of orbits started with the same total energy to map out the orbit structure of a portion of phase-space. How should we choose the initial conditions for our grid of orbits? We need to set the 6 phase-space coordinates for each orbit. Requiring that they have the same energy gives us 1 constraint. To further reduce the dimensionality, we will assume $y=v_x=v_z=0$ (we now have 4 constraints). We will then choose a grid in $x, z$ to set the final two coordinates. At any location in our $x,z$ grid, we will use the energy to determine the value of $v_y$ from:

$$
E = \frac{1}{2}(v_x^2 + v_y^2 + v_z^2) + \Phi(x,y,z)\\
v_y = \sqrt{2\,(E - \Phi(x, 0, z))}
$$

For the energy, we will use $E = 0.195~\textrm{kpc}^2~\textrm{Myr}^{-2}$:

In [ ]:
# (No modifications needed here)
grid_E = 0.195 * (u.kpc / u.Myr) ** 2

Generate a 1D grid of 41 $x$ values between $(15, 25)~\textrm{kpc}$, and a 1D grid of 41 $z$ values between $(0, 20)~\textrm{kpc}$. Use these 1D grids to construct a 2D grid with all 1,681 pairs of coordinates (*Hint: use `numpy.meshgrid()`*). Store an array of all $x,y,z$ values (all $y$ values are 0) in the variable `grid_pos`:

In [ ]:
_x_grid = np.linspace(15, 25, 41)
_z_grid = np.linspace(0, 20, 41)
grid_shape = (len(_x_grid), len(_z_grid))

# x_grid, z_grid = ...

Compute the potential energy at all locations in the grid, and use the difference of the grid energy `grid_E` and the potential energy to compute the initial $v_y$:

In [ ]:
# grid_Phi = ...
# vy_grid = ...

(Some of the $v_y$ values may come out as NaN: that is ok, you can ignore those - there are some values of our $x,z$ grid that are outside of the iso-potential-energy surface)

Plot the grid of $x,y$ positions colored by the value of $v_y$ at each location (i.e. the following cell should execute)

In [ ]:
# (No modifications needed here)
plt.figure(figsize=(7, 6))
plt.pcolormesh(x_grid, z_grid, vy_grid.reshape(grid_shape).to_value(u.km / u.s))
plt.xlabel("$x_0$")
plt.ylabel("$z_0$")

cb = plt.colorbar()
cb.set_label(r"$v_y$")

Set up the full grid of initial conditions as a `PhaseSpacePosition` object named `grid_w0`:

In [ ]:
# (No modifications needed here)
grid_vel = np.zeros(grid_pos.shape) * u.km / u.s
grid_vel[1] = vy_grid

grid_w0 = gd.PhaseSpacePosition(pos=grid_pos * u.kpc, vel=grid_vel)

Compute the orbits for all of the initial conditions in the grid using the default `LeapfrogIntegrator`, using a timestep of 2 Myr, and integrate for 10 Gyr

In [ ]:
# grid_orbits = ...

Compute the angular momentum components for all orbits, and then compute the peak-to-peak spread in each angular momentum component for each orbit (i.e. compute $\textrm{max}(L_i) - \textrm{min}(L_i)$ for each component $i$ for each orbit)

In [ ]:
# grid_orbits_L = ...
# ptp_L = ...

Make a 3 panel plot (panels corresponding to the 3 angular momentum components) showing a 2D image of the peak-to-peak spread in each component (i.e. the plot commands below should execute)

* What structure do you see in this diagram? 
* What do you think causes the structure we see in this diagram?
* Can you identify the transition from tube to box orbits?

## Irregular and Chaotic Orbits

A final class of orbits that are found in very flattened axisymmetric potentials and more frequently in triaxial potentials are known as *chaotic* orbits. There is a lot to say about chaos, but one way of thinking of chaotic orbits are that they are special orbits in which the actions do not exist: only energy is conserved (in time-independent potentials with chaotic orbits). The fact that chaotic orbits no longer have the three actions as isolating integrals of motion means that they are free to explore a much larger volume in phase-space. Recall that regular orbits exist in 6D phase-space, but the existence of three actions limits orbits (with particular values for their actions) to exploring a 3D manifold embedded within the 6D phase-space. Orbits that only conserve energy as an integral of motion can instead explore a 5D manifold known as the *energy hypersurface*. 

Chaotic orbits also look more erratic: Regular orbits tend to have a smooth, well-behaved shape to their trajectories. 

### Example: Comparing the morphology of regular and chaotic orbits

As a demonstration of the visual differences between regular and chaotic orbits, we will use a triaxial Navarro-Frenk-White potential model to compute two orbits:

In [ ]:
triaxial_nfw = gp.NFWPotential.from_circular_velocity(
    v_c=230 * u.km / u.s, r_s=5 * u.kpc, a=1, b=0.9, c=0.8, units=galactic
)

Compare the two orbits below -- which one do you think is chaotic?

In [ ]:
regular_chaotic_w0 = gd.PhaseSpacePosition(
    pos=([[0, 20, 0], [0.1, 15, 0]] * u.kpc).T,
    vel=([[15, 0, 120], [5.0, 0.0, 230]] * u.km / u.s).T,
)

regular_chaotic_orbits = triaxial_nfw.integrate_orbit(
    regular_chaotic_w0, dt=0.5, t1=0, t2=10 * u.Gyr
)

for n in range(regular_chaotic_orbits.norbits):
    _ = regular_chaotic_orbits[:, n].plot()

### Chaotic Timescales

Not all chaotic orbits obviously appear chaotic from their morphologies. This is often because some orbits are only *weakly chaotic* whereas others are *strongly chaotic*. There is no global definition to strong and weak chaos because it depends on the relevant timescales of whatever system you are looking at: Weakly chaotic orbits have a chaotic timescale much smaller than the dynamical time, whereas strongly chaotic orbits have comparable timescales. In the case of weakly chaotic orbits, the orbits may look and behave like nearby regular orbits for a long time (i.e. for all astronomically-relevant timescales). In the case of strongly chaotic orbits, the morphology of the orbit changes on the order of an orbital timescale, like in the example above.

One way of quantifying the strength of chaos for an orbit is with the *Maximum Lyapunov Exponent* or the inverse, sometimes called the *Lyapunov Time*. The MLE is a measure of how quickly two infinitesimally-close orbits diverge from one another. We will not go through the math to derive the MLE here (see section 3.7.3 c in Binney and Tremaine). However, I will note that you can compute the MLE in Gala using the `gala.dynamics.fast_lyapunov_max()` function. For example, for the two orbits shown above:

In [ ]:
lyap1 = gd.fast_lyapunov_max(
    regular_chaotic_w0[0], triaxial_nfw, dt=2.0, n_steps=200_000, return_orbit=False
)

In [ ]:
lyap2 = gd.fast_lyapunov_max(
    regular_chaotic_w0[1], triaxial_nfw, dt=2.0, n_steps=200_000, return_orbit=False
)

For regular orbits, the estimate of the Lyapunov exponent continues decreasing as the integration time increases. However for chaotic orbits, the value eventually saturates to a number: This is the estimate of the MLE.

Let's plot the time series estimate of the LE for the two orbits above:

In [ ]:
plt.loglog(lyap1[:, 0])
plt.loglog(lyap2[:, 0])
plt.xlabel("timestep")
plt.ylabel("Lyapunov exponent")

We can see that the first orbit, which visually looks erratic, has a finite MLE that saturates to a value around $4\times 10^{-4}~\textrm{Myr^{-1}}$, corresponding to a Lyapunov timescale of about $2.5~\textrm{Gyr}$.

There are many other interesting aspects of chaotic orbits, but we will leave that to be covered in discussions (if you get to this point, ask the workshop coordinators about chaos :-)).